In [16]:
## Load packages and data

import pandas as pd
import numpy as np
import re 
import datetime as dt


%store -r merged_df_demo_abc
%store -r not_merged_df_demo_abc

In [17]:
## Select columns we want to focus on
## only keeping the columns we want

columns_to_keep = ["Local ID", 'Region', "Date of birth", "Gender", "Race", "Ethnicity", 
                   'Irritability/Agitation', 'Date Reviewed']

merged_df_demo_abc_selected = merged_df_demo_abc[columns_to_keep].copy()
merged_df_demo_abc_selected.dtypes

merged_df_demo_abc_selected['Local ID'] = merged_df_demo_abc_selected['Local ID'].astype(str)

%store merged_df_demo_abc_selected

Stored 'merged_df_demo_abc_selected' (DataFrame)


In [18]:
## Select only dates that were entered in correctly based on length
merged_df_demo_abc_selected["Date Reviewed"].str.len().unique()

merged_df_demo_abc_selected["date_length"] = merged_df_demo_abc_selected["Date Reviewed"].astype(str).str.len()
print(merged_df_demo_abc_selected["date_length"].value_counts())

# Show the number of rows in which the value of Date Reviewed is misformatted - 19 is most popular
print(merged_df_demo_abc_selected[merged_df_demo_abc_selected['date_length']!=19].shape)

## Only keep correctly formatted dates
merged_date_cleaned = merged_df_demo_abc_selected[merged_df_demo_abc_selected['date_length']==19].reset_index()
merged_date_cleaned.shape

merged_date_cleaned['date_reviewed'] = pd.to_datetime(merged_date_cleaned['Date Reviewed'])


19    19425
3        32
10        4
Name: date_length, dtype: int64
(36, 9)


In [19]:
## Subset to beginning of 2018 to leave out mis-entered date (like 2012 for person born after 2012)

merged_date_cleaned_new = merged_date_cleaned[merged_date_cleaned['date_reviewed'] > 
                                          pd.to_datetime('2018-01-01')].reset_index()

merged_date_cleaned_new.shape


(16241, 12)

In [20]:
## Clean race

merged_date_cleaned_new['race'] = merged_date_cleaned_new.Race.str.lower()

## Regex to detect when multiple races are listed
biracial_pattern = r'.*and.*|.*\/.*|.*,.*|other: biracial'

## Find original 'Race' categories that are multiracial
biracial_findall = [race for race in merged_date_cleaned_new.race if re.findall(biracial_pattern, race)]

## Categorize remaining multiracial categories as 'other'
merged_date_cleaned_new['race_clean'] = np.where((merged_date_cleaned_new.race.isin(biracial_findall)) |
                                                (merged_date_cleaned_new.race.isin(['other: mixed race',
                                                                                  'other: bi-racial',
                                                                                  'other: hispanic - puerto rican'])),
                                                'other', merged_date_cleaned_new.race)

## Clean to white non-hispanic, black non-hispanic, hispanic (any), other -- use str.contains
## Set as Hispanic to make sure 'white' is white non-Hispanic
race_dict = [(merged_date_cleaned_new.race.str.contains('white')) & 
             (merged_date_cleaned_new.race != 'other: white, hispanic') &
             (merged_date_cleaned_new.Ethnicity == 'Not of Hispanic origin') & 
             (merged_date_cleaned_new.race_clean != 'other') | 
             (merged_date_cleaned_new.race == 'other: caucasian') & 
             (merged_date_cleaned_new.Ethnicity == 'Not of Hispanic origin'),
             (merged_date_cleaned_new.race.str.contains('black or african american')) & 
             (merged_date_cleaned_new.Ethnicity == 'Not of Hispanic origin') & 
             (merged_date_cleaned_new.race_clean != 'other'), 
             (merged_date_cleaned_new.race.str.contains('hispanic')) & 
             (merged_date_cleaned_new.Ethnicity == 'Hispanic - specific origin not specified') &
             (merged_date_cleaned_new.race_clean != 'other')| 
             (merged_date_cleaned_new.race.isin(['other: latina', 'other: latino'])) & 
             (merged_date_cleaned_new.Ethnicity == 'Hispanic - specific origin not specified') & 
             (merged_date_cleaned_new.race_clean != 'other')]
              

race_codeto = ['white_non_hispanic','black_non_hispanic', 'hispanic']

merged_date_cleaned_new['race_clean'] = np.select(race_dict, race_codeto, default = 'other')

## Create binary variable for regression
merged_date_cleaned_new['is_black'] = merged_date_cleaned_new['race_clean'] == 'black_non_hispanic'
                                                                                
## Show count
merged_date_cleaned_new.race_clean.value_counts()

white_non_hispanic    8400
other                 3915
black_non_hispanic    3081
hispanic               845
Name: race_clean, dtype: int64

In [21]:
## Clean gender

merged_date_cleaned_new['gender_clean'] = np.where(merged_date_cleaned_new.Gender.str.contains('Other'), 'Other', merged_date_cleaned_new.Gender)

## Create binary variable for regression
merged_date_cleaned_new['is_male'] = merged_date_cleaned_new['gender_clean'] == 'Male'

merged_date_cleaned_new.gender_clean.value_counts()

Male      10984
Female     5205
Other        52
Name: gender_clean, dtype: int64

In [22]:
## Drop rows where the irritability score is invalid

merged_date_cleaned_new['irrit_score'] = merged_date_cleaned_new['Irritability/Agitation'].copy()
print(merged_date_cleaned_new[merged_date_cleaned_new['irrit_score'].isna()].shape)

merged_date_cleaned_new = merged_date_cleaned_new[~merged_date_cleaned_new['irrit_score'].isna()]

(30, 18)


In [23]:
## Get black and white only

bw_only = merged_date_cleaned_new[merged_date_cleaned_new['race_clean'].isin(['black_non_hispanic', 'white_non_hispanic'])].copy()


In [24]:
## Get male and female only

mf_only = merged_date_cleaned_new[merged_date_cleaned_new['gender_clean'].isin(['Male', 'Female'])].copy()


In [25]:
%store merged_date_cleaned_new
%store bw_only
%store mf_only

Stored 'merged_date_cleaned_new' (DataFrame)
Stored 'bw_only' (DataFrame)
Stored 'mf_only' (DataFrame)
